# Homework: Phrasal Chunking

## Overview

In this assignment we mainly explored and trained different LSTM networks to obtain as high as 96.00 F1 score at given test set.

## Embedding

A single input consists of a word itself and its POS tag in the sentence. We embed words and POS tags into vectors separately and then concatenate them to form a single input.

### POS Tag Embedding

With all POS tags and a special one <UNKNOWN>, we build a dictionary and give each POS tag an index. Then we use torch.nn.Embedding() to embed different POS tags to vectors of an assigned size.

### Word Embedding

We use ELMo embeddings as pre-trained token-level word representations. Each word is mapped to a ELMo vector of size 1024. Words with similar meaning (syntactically and semantically) will have vector representations that are close together.


## BiLSTM

LSTM is an RNN architecture that takes sequential input (words in a sentence and their POS tags) and produces sequential output (chunk tags), and is capable of learning long-term dependencies. With BiLSTM, sequential input is scanned both from the beginning to the end and from the end to the beginning. The idea behind it is that both the context before and the context after can influence the chunk tag of a word.

At each timestamp t, we have $h_t^f$/$h_t^b$ that is computed forwards/backwards. The output is then computed with a linear function of $h_t^f$ and $h_t^b$.


## BiLSTM-CRF 

We implemented a BiLSTM-CRF network proposed in  [Huang, Xu, and Yu 2015]. This network has an additional CRF layer on top of the BiLSTM layer. The CRF has a global transition score from one tag to another and an emission score obtained from the BiLSTM outputs. We denote $A _ { y _ { i } , y _ { j } }$ as the transition score from tag $y_{i}j$ to tag $y_j$ and $P _ { i , y _ { i } } as the emission score from input word i to tag yi. The score of a input sequence X and tag sequence y is:

$s ( \mathbf { X } , \mathbf { y } ) = \sum _ { i = 0 } ^ { n } A _ { y _ { i+1 } , y _ { i } } + \sum _ { i = 1 } ^ { n } P _ { i , y _ { i } }$

We use the negative log likelihood as the loss for training. The likelihood is defined as: 

$\frac { e ^ { s ( \mathbf { X } , \mathbf { y } ) } } { \sum _ { \widetilde { \mathbf { y } } \in \mathbf { Y } _ { \mathbf { X } } } e ^ { s ( \mathbf { X } , \widetilde { \mathbf { y } } ) } }$

After adding the additional CRF layer on vanilla BiLSTM network We obtained the best score of 95.84.


## BiLSTM-CRF with encoder

We further added an encoder to initialize initial hidden state of LSTM. We use the same BiLSTM as an encoder and feed the same input to the encoder. We take the hidden state of the encoder as the initial hidden state of the BiLSTM. 

After adding the encoder, we improved the best score to 96.00.


## Utilizing features in the network

From data sets we can extract some features, including word features, spelling features, and context features. (Huang et al.)

As mentioned above, for word features we utilized ELMo word embeddings, where each word is transformed to a vector of 1024 length.

We noticed that bigram features (which is one of context feature) were provided for us in this assignment. So we utilized those features as inputs of our BiLSTM-CRF.

There are 20 bigram features provided in `*.feats.gz` files, we made use of the first 19 features. For each feature we use a embedding layer (`torch.nn.embedding  `) to transform the feature into a shorter vector. (In our experiments we use length of 32 vector for each individual feature.) Then concatenate the vectors of all features together with the word embeddings as the input of Bi-LSTM networks.

Along with bigram features, we do similar operations to the part-of-speech tags as features as inputs.

In our experiments of using bigram features, we got best F1 score of around 95.7 in epoch 9 in BiLSTM-CRF with encoder, which does not have much improvement on F1 score. Our analysis is that in LSTM, the context features are already stored as states inside the networks, adding bigram features is kind of redundant and does not have much effects.


## Experiment settings

We explored several different hyperparameters: number of LSTM layers, hidden dimensions, learning rates and optimizers. **we obtained our best result with 2 layers, 600 hidden  dimensions, learning rate 0.01 and SGD optimizer which gives F1 score 96.00**. We found that the result is generally not sensitive to the number of layers and hidden dimensions. Since we are using SGD and we don’t have much training data, a relatively small learning of 0.01 would converge to a better optima as compared to 0.1 which is used in [Huang, Xu, and Yu 2015]. We also tried several different optimizers and found that SGD is usually better than others, such as Adam and SGD with momentum. 

We trained the model for 20 epochs, which takes roughly 2 hours on GTX 1080 ti GPU. We performed validation on dev set and obtained the best model after 13 epochs.


## Reference:

[Huang, Xu, and Yu 2015] Huang, Z.; Xu, W.; and Yu, K. 2015. Bidirectional lstm-crf models for sequence tagging.arXiv preprint arXiv:1508.01991.


In [14]:
import sys
import os
import perc
from bilstmcrf.util import *
from bilstmcrf.BiLSTM_CRF import BiLSTM_CRF, BiLSTM_Enc_Dec_CRF

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tagset = perc.read_tagset("data/tagset.txt")

print("reading testing data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)

print('loading model...', file=sys.stderr)
model_data = load_model("h600layer2lr0.01enc.model")
word_idx = model_data['word_index']
speech_tag_idx = model_data['speech_tag_index']
tag2idx = model_data['tag_index']
idx2tag = model_data['reverse_tag_index']
layer = 2
hidden = 600
pos_dim = 64
model = BiLSTM_Enc_Dec_CRF(len(speech_tag_idx), len(tag2idx), device, layer, hidden, pos_dim)
model.load_state_dict(model_data['model'])
model.to(device)
print('done.', file=sys.stderr)

print('preparing testing data...', file=sys.stderr)
test_tuples = prepare_test_data(test_data, speech_tag_idx)
print('done.', file=sys.stderr)

predicted_tags = test_model(model, test_tuples, idx2tag, device)

output = format_prediction(predicted_tags, test_data)

reading testing data ...
done.
loading model...
done.
preparing testing data...
initializing ELMo embedding... 
loaded. 
100%|██████████| 500/500 [00:18<00:00, 26.47it/s]done.


In [15]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(output, boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5809; correct phrases: 5564
             ADJP: precision:  78.00%; recall:  78.79%; F1:  78.39; found:    100; correct:     99
             ADVP: precision:  76.71%; recall:  83.17%; F1:  79.81; found:    219; correct:    202
            CONJP: precision: 100.00%; recall:  40.00%; F1:  57.14; found:      2; correct:      5
             INTJ: precision: 100.00%; recall: 100.00%; F1: 100.00; found:      1; correct:      1
               NP: precision:  97.12%; recall:  97.12%; F1:  97.12; found:   3026; correct:   3026
               PP: precision:  97.66%; recall:  98.94%; F1:  98.29; found:   1237; correct:   1221
              PRT: precision:  83.33%; recall:  68.18%; F1:  75.00; found:     18; correct:     22
             SBAR: precision:  89.81%; recall:  90.65%; F1:  90.23; found:    108; correct:    107
               VP: precision:  96.17%; recall:  96.00%; F1:  96.09; found:   1098; correct:   1100
accura